1.Sudoku Solver
-
Generation of Sudoku:
-------------------------

Firstly, the random generation of empty slots of the soduku are done by taking into consideration of possible elements by avoiding conflicts with row, column, and 3x3 boxes.

If our set becomes empty then only row conflicts are taken into consideration and randomly elements are generated

In [37]:
import random 
def generate_sudoku(sudoku):
    a = [[] for i in range(9)]
    for i in range(9):
        a[i] = list(map(int,sudoku[i]))
    # print(a)
    for i in range(9):
        for j in range(9):
            if a[i][j]==0:
                s = set()
                for k in range(9):
                    if i!=k:
                        s.add(a[k][j])
                    if j!=k:
                        s.add(a[i][k])
                x = (i//3)*3
                y = (j//3)*3
                for k in range(3):
                    s.update([a[x+k][y],a[x][y+k],a[x+k][y+k]])
                s.update([a[x+2][y+1],a[x+1][y+2]])
                s = {1,2,3,4,5,6,7,8,9}-s
                s = list(s)
                if len(s)==0:
                    s = set()
                    for k in range(9):
                        s.add(a[i][k])
                    s = {1,2,3,4,5,6,7,8,9}-s
                    s = list(s)
                a[i][j] = s[random.randint(0, len(s)-1)]
    return a 

Fitness Function:
----------------

The Fitness of the sudoku I have taken is sum of total no of distinct elements in the row, column, 3x3 boxes of the sudoku.
A better Sudoku is the one having more Fitness value in this case

Therefore this Genetic Algorithm is to increase the Fitness Values of Sudokus

In [38]:
import numpy as np
def fitness(sudoku):
    def row(sudoku):
        fit = 0
        for i in range(9):
            fit+=len(set(sudoku[i]))
        return fit
    def col(sudoku):
        fit = 0
        dup = sudoku.copy()
        dup = np.array(dup)
        dup = dup.transpose()
        for i in range(9):
            fit+=len(set(dup[i]))
        return fit
    def box_3x3(sudoku):
        fit = 0
        for i in range(0,9,3):
            s = set()
            for j in range(0,9,3):
                s.update([sudoku[i][j],sudoku[i+1][j],sudoku[i+2][j],sudoku[i][j+1],sudoku[i+1][j+1],sudoku[i+2][j+1],sudoku[i][j+2],sudoku[i+1]                          [j+2],sudoku[i+2][j+2]])
                fit+=len(s)
        return fit
    return row(sudoku)+col(sudoku)+box_3x3(sudoku)

In [39]:
def fit_list(sudoku_list):
    fit = [fitness(sudoku_list[i]) for i in range(10)]
    return fit

Cross Over:
------------

The Crossover is done by swapping two sudokus half portion on either sides.

This is similar to what most of the Genetic Algorithms do. i.e. Crossover through mid-point

In [40]:
def cross_over(sudoku1,sudoku2):
    import random
    # print(sudoku1,sudoku2)
    x, y = sudoku1[:5]+sudoku2[5:], sudoku2[:5]+sudoku1[5:]
    # print(x,y)
    return [x,y]

Selection :
------------

Selection is done randomly, a die is thrown and based on fitness values two sudokus are selected for crossover.


In [41]:
def selection(fitnesses):
    import random
    selects = []
    for _ in range(5):
        rand_no = random.randint(0,50)
        for i in range(10):
            if rand_no<=fitnesses[i][1]:
                selects.append(fitnesses[i][0])
                break
            rand_no-=fitnesses[i][1]
        last = selects[-1]
        while(selects[-1]==last):
            rand_no = random.randint(0,99)
            for i in range(10):
                if rand_no<=fitnesses[i][1]:
                    last = fitnesses[i][0]
                    break
                rand_no-=fitnesses[i][1]
        selects.append(last)
    return selects


Mutation:
-----------

Mutation here is also random, a random row is generated again by the help of generate sudoku function.  

In [42]:
def mutation(sudoku,original):
    import random
    index = random.randint(0,10)
    if index<9:
        sudoku[index] = list(map(int, original[index]))
        i = index
        for j in range(9):
            if sudoku[i][j]==0:
                s = set()
                for k in range(9):
                    if i!=k:
                        s.add(sudoku[k][j])
                    if j!=k:
                        s.add(sudoku[i][k])
                x = (i//3)*3
                y = (j//3)*3
                for k in range(3):
                    s.update([sudoku[x+k][y],sudoku[x][y+k],sudoku[x+k][y+k]])
                s.update([sudoku[x+2][y+1],sudoku[x+1][y+2]])
                s = {1,2,3,4,5,6,7,8,9}-s
                s = list(s)
                if len(s)==0:
                    s = set()
                    for k in range(9):
                        s.add(sudoku[i][k])
                    s = {1,2,3,4,5,6,7,8,9}-s
                    s = list(s)
                sudoku[i][j] = s[random.randint(0, len(s)-1)]
    return sudoku

This function is to print best 5 sudokus in a clear format

In [43]:
def print_sudokus(sudokus_list):
    from IPython.display import display, HTML
    display(HTML("<h1>BEST 5 SUDOKUS -></h1>"))
    for k in range(5):
        display(HTML(f"<h1>sudoku {k+1}</h1>"))
        if fitness(sudokus_list[k])==243:
            display(HTML("<h2>SOLUTION:</h2>"))
        for i in range(9):
            string="<tr><td>" + "</td><td>".join(str(sudokus_list[k][i][:]).split()) + "</td></tr>"
            display(HTML(f"<table>{string}</table>"))
        print()

Initial Sudoku:
---------------

In [44]:
sudoku = [ "080000090",
           "007502800",
           "600807005",
           "370080051",
           "200000008",
           "950040032",
           "800104009",
           "001903600",
           "040000020" ]

Generating 10 sudokus
-----------------------

In [45]:
sudokus_list = []
for i in range(10):
    sudokus_list.append(generate_sudoku(sudoku))

Exploring Genetic Algorithm for 1000 Times
--------------------------------------------

In [46]:
for i in range(1000):
    fitness_values = fit_list(sudokus_list)
    fitnesses = [[i+1,fitness_values[i]] for i in range(10)]
    fitnesses.sort(key=lambda x:-x[1])
    selection_values = [round((fitnesses[i][1]*100)/sum(fitness_values)) for i in range(10)]
    for i in range(10):
        fitnesses[i][1] = selection_values[i]
    selects = selection(fitnesses)
    for i in range(0,10,2):
        sudokus_list[i],sudokus_list[i+1] = cross_over(sudokus_list[i],sudokus_list[i+1])
        sudokus_list[i] = mutation(sudokus_list[i],sudoku)
        sudokus_list[i+1] = mutation(sudokus_list[i+1],sudoku)
    if 243 in fit_list(sudokus_list):
        break

Sorting Sudokus based on fitness values
-----------------------------------------

In [47]:
sudokus_list.sort(key=lambda x : fitness(x),reverse=True)

Printing best 5 sudokus
-----------------------

In [48]:
print_sudokus(sudokus_list)

"[1,","8,","5,","4,","3,","6,","2,","9,",7]


"[4,","3,","7,","5,","9,","2,","8,","1,",6]


"[6,","9,","2,","8,","1,","7,","3,","4,",5]


"[3,","7,","6,","2,","8,","9,","4,","5,",1]


"[2,","1,","4,","3,","7,","5,","9,","6,",8]


...

"[2,","1,","4,","3,","7,","5,","9,","6,",8]


"[9,","5,","8,","1,","4,","6,","7,","3,",2]


"[8,","6,","3,","1,","2,","4,","5,","7,",9]


"[5,","2,","1,","9,","4,","3,","6,","8,",7]


"[7,","4,","9,","6,","5,","8,","1,","2,",3]
